face recognition from Live Videos!

In [1]:
import random
import face_recognition
import os, sys
import cv2
import numpy as np
import math
import ctypes
from screeninfo import get_monitors

In [2]:
def face_confidence(face_distance, face_match_threshold=0.6):
    range = (1.0 - face_match_threshold)
    linear_val = (1.0 -face_distance) / (range * 2.0)

    if face_distance > face_match_threshold:
        return str(round(linear_val * 100, 2 )) + '%'
    else:
        value = (linear_val + ((1.0 - linear_val) * math.pow((linear_val - 0.5) * 2, 0.2))) * 100
        return str(round(value, 2)) + '%'

In [5]:
class FaceRecognition:
    face_locations = []
    face_encodings = []
    face_names = []
    known_face_encodings =[]
    known_face_names = []
    process_current_frame = True

    def __init__(self):
        self.encode_faces()
        
    def encode_faces(self):
        data_path = './all_faces'
        for image in os.listdir(data_path):
            face_image = face_recognition.load_image_file( data_path + f'/{image}') # an image of a person's face
            face_encoding = face_recognition.face_encodings(face_image)
            if len(face_encoding) > 0:
                face_encoding = face_encoding[0] 
                self.known_face_encodings.append(face_encoding)
                self.known_face_names.append(image)

        print("Number of People: " + str(len(self.known_face_names)) + "\n")
        print(self.known_face_names)
    
    def run_recognition(self):
        video_capture = cv2.VideoCapture(0)
        monitors = get_monitors()
        screen_width = monitors[0].width
        screen_height = monitors[0].height
        video_capture.set(cv2.CAP_PROP_FRAME_WIDTH, screen_width)
        video_capture.set(cv2.CAP_PROP_FRAME_HEIGHT, screen_height)
        aspect_ratio = screen_width / screen_height
        new_width = int(screen_height * aspect_ratio)
        
        if not video_capture.isOpened():
            sys.exit('Video source was not found') # Video Capture Failed!

        while True:
            ret, frame = video_capture.read()

            if self.process_current_frame:
                # resize and change the frame to RGB
                small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
                rgb_small_frame = cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB)

                # Find all faces in the current frame
                self.face_locations = face_recognition.face_locations(rgb_small_frame)
                self.face_encodings = face_recognition.face_encodings(rgb_small_frame, self.face_locations)

                self.face_names = []
                for face_encoding in self.face_encodings:
                    matches = face_recognition.compare_faces(self.known_face_encodings, face_encoding)
                    name = 'Not Atar!'
                    confidence = ''

                    face_distances = face_recognition.face_distance(self.known_face_encodings, face_encoding)
                    best_match_index = np.argmin(face_distances)

                    if matches[best_match_index]:
                        name = self.known_face_names[best_match_index]
                        name = name.split(".jpg")[0]
                        confidence = face_confidence(face_distances[best_match_index])

                    self.face_names.append(f'{name} ({confidence})')

            self.process_current_frame = not self.process_current_frame

            # Display annotations
            for (top, right, bottom, left), name in zip(self.face_locations, self.face_names):
                top *= 6
                right *= 6
                bottom *= 6
                left *= 6
                
                frame = cv2.resize(frame, (new_width, screen_height)) # resized frame
            
                cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
                cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), -1)
                cv2.putText(frame, name, (left + 6, bottom - 6), cv2.FONT_HERSHEY_DUPLEX, 0.8, (255, 255,255), 1)
           
            cv2.imshow('Face Booth', frame)
            if cv2.waitKey(1) == ord('q'):
                break

        video_capture.release()
        cv2.destroyAllWindows()

In [6]:
# Run the Face Recognition application
fr = FaceRecognition()
fr.run_recognition()

Number of People: 33

['almog.jpg', 'alon.jpg', 'amro.jpg', 'atar.jpg', 'eilam.jpg', 'guy.jpg', 'hind.jpg', 'itay.jpg', 'jake.jpg', 'lia.jpg', 'Maisan.jpg', 'Margalit.jpg', 'matan.jpg', 'michael.jpg', 'michal.jpg', 'mirvat.jpg', 'MohammedE.jpg', 'MohammedS.jpg', 'mohammed_a.f.jpg', 'Mohanad.jpg', 'nitai.jpg', 'NoamBoker.jpg', 'NoamDavid.jpg', 'ofir.jpg', 'saar.jpg', 'saleh.jpg', 'ShaharH.jpg', 'ShaharM.jpg', 'tamar.jpg', 'Tamir.jpg', 'uri.jpg', 'uthman.jpg', 'yifat.jpg']
